<a href="https://colab.research.google.com/github/dineshRaja29/TextEncodings/blob/main/textEncodings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#<font color = 'green'><b>INTRODUCTION


We will explore various ways to encode text (conversion of text into numeric vector/scaler) from classical to state of art approach. There are broadly four methods which are extensively used in ML/DL.



- BoW using SkLearn + Sparse Matrices
- TF-IDF using SkLearn
- Word2Vec using Gensim
- BERT using bert-serving library + APIs.




#<font color = 'green'><b>Data loading

In [6]:
import shutil
import os
import sys
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [7]:
print(sys.version)

3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [8]:
# pre-requisted for stopword and word_tokenize
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
# pre-requisted for Lemmatize
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [10]:
# Source: http://ai.stanford.edu/~amaas/data/sentiment/
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2022-05-03 18:16:57--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  31.2MB/s    in 2.6s    

2022-05-03 18:17:00 (31.2 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [11]:
# uncompress and see the data
! ls
! pwd

aclImdb_v1.tar.gz  sample_data		    uncased_L-12_H-768_A-12.zip
out.file	   uncased_L-12_H-768_A-12
/content


In [12]:
! tar -zxvf aclImdb_v1.tar.gz >& /dev/null

In [13]:
! ls

aclImdb		   out.file	uncased_L-12_H-768_A-12
aclImdb_v1.tar.gz  sample_data	uncased_L-12_H-768_A-12.zip


In [ ]:
#Google: "Unzip tar gz file colab" ----> https://stackoverflow.com/questions/49685924/extract-google-drive-zip-from-google-colab-notebook
# shutil.unpack_archive("/content/aclImdb_v1.tar.gz", "/content/")
# ! ls /content/

In [14]:
! ls -l /content/aclImdb

total 1724
-rw-r--r-- 1 7297 1000 903029 Jun 11  2011 imdbEr.txt
-rw-r--r-- 1 7297 1000 845980 Apr 12  2011 imdb.vocab
-rw-r--r-- 1 7297 1000   4037 Jun 26  2011 README
drwxr-xr-x 4 7297 1000   4096 May  3 18:17 test
drwxr-xr-x 5 7297 1000   4096 May  3 18:17 train


In [15]:
! head -10 /content/aclImdb/imdb.vocab

the
and
a
of
to
is
it
in
i
this


In [16]:
! ls /content/aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [17]:
! ls /content/aclImdb/train/pos | head -10

0_9.txt
10000_8.txt
10001_10.txt
10002_7.txt
10003_8.txt
10004_8.txt
10005_7.txt
10006_7.txt
10007_7.txt
10008_7.txt


In [19]:
! cat /content/aclImdb/train/pos/6250_10.txt | tr -s ' ' '\n' | head -20 | tr -s '\n' ' '

There is something special about the Austrian movies not only by Seidl, but by Spielmann and other directors as well. 

In [20]:
# load data from k-reviews from imdb/train/pos reviews

k=100

raw_data = [] # empty list, 


index_file = dict(); # store mapping from index to filename

directory = r'/content/aclImdb/train/pos/'

i=0

for f in os.listdir(directory): # for each file in the subfolder
      
  if f.endswith(".txt"): # check for text file
    fname = directory + "/" + f
    
    tmp = open(fname, "r") # read file 

    raw_data.append(tmp.read())
    index_file[i] = fname
        
    i += 1

    if i==k: # read k files
      break


In [21]:
print(i)
print(index_file)
print(raw_data[0])

100
{0: '/content/aclImdb/train/pos//11525_8.txt', 1: '/content/aclImdb/train/pos//8932_9.txt', 2: '/content/aclImdb/train/pos//8305_7.txt', 3: '/content/aclImdb/train/pos//11568_7.txt', 4: '/content/aclImdb/train/pos//10599_8.txt', 5: '/content/aclImdb/train/pos//11901_9.txt', 6: '/content/aclImdb/train/pos//343_10.txt', 7: '/content/aclImdb/train/pos//11378_8.txt', 8: '/content/aclImdb/train/pos//883_9.txt', 9: '/content/aclImdb/train/pos//10406_10.txt', 10: '/content/aclImdb/train/pos//11699_10.txt', 11: '/content/aclImdb/train/pos//3863_7.txt', 12: '/content/aclImdb/train/pos//11289_10.txt', 13: '/content/aclImdb/train/pos//10783_10.txt', 14: '/content/aclImdb/train/pos//5443_8.txt', 15: '/content/aclImdb/train/pos//3689_8.txt', 16: '/content/aclImdb/train/pos//11065_8.txt', 17: '/content/aclImdb/train/pos//6351_8.txt', 18: '/content/aclImdb/train/pos//9321_8.txt', 19: '/content/aclImdb/train/pos//672_9.txt', 20: '/content/aclImdb/train/pos//4989_8.txt', 21: '/content/aclImdb/train

##<font color = 'green'><b> Data preprocessing

Not always removing stopword is helpful, 'not' is considered as a stopword
for example:
- He is bad boy
- He is not a bad boy <br>
 After removal of stopword both makes the same sentences thus, harmful here <br>

 Another approach to fix this is to filter the list of stopwords based on language which you are working


In [22]:

example_sent = "This is a sample sentence, showing off the stop words filtration."

def stopWordRemoval(sent, stop_words):
  word_tokens = word_tokenize(sent) # tokenize
  filtered_sentence = "";
  for w in word_tokens:
      if w not in stop_words:
          filtered_sentence += " " + w
  return filtered_sentence

stop_words = set(stopwords.words('english')) # NLTK 
print(stop_words)

print(raw_data[0])
print(stopWordRemoval(raw_data[0], stop_words) ) # call the function

{'before', 'our', 'just', 'their', 'me', 'you', 'him', 'all', 'how', 'whom', 'themselves', 'what', 'for', 'then', 'wouldn', 'where', 'your', 'them', 'if', 'that', 'very', "hadn't", 'we', 'does', 'both', 'been', 'no', 'is', 'any', 'shouldn', 'during', 'which', 'these', 'the', 'do', "weren't", 'but', 'after', 'with', 'few', 'a', 'below', "you'll", 'should', 'again', 'myself', 'when', 't', 'such', 'hers', 'her', 'too', "you've", 'most', 'by', 'further', 'through', 'off', 'had', 're', 'being', 'be', 'each', 'under', 'ma', 'couldn', 'not', "you'd", 'doesn', 'i', 'as', "needn't", "don't", 'shan', 'while', 'o', 'doing', "doesn't", 'they', 'have', 'out', "it's", 'why', 'aren', 'she', 'my', 'and', 'to', "should've", 'll', 'wasn', "mustn't", "couldn't", "you're", "wouldn't", 'between', 'has', 'his', 'now', 'there', "won't", 'was', 'are', 'who', 'or', 'at', 'yours', 'than', 'some', 'weren', 'because', 'here', 'y', 'about', 's', "mightn't", 'herself', "wasn't", 'yourself', 'its', 'having', 'mustn'

In [23]:
# Lemmatize
# Source: https://pythonprogramming.net/lemmatizing-nltk-tutorial/

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize("cats"))
print(lemmatizer.lemmatize("cacti"))
print(lemmatizer.lemmatize("geese"))
print(lemmatizer.lemmatize("rocks"))
print(lemmatizer.lemmatize("python"))
print(lemmatizer.lemmatize("better", pos="a"))
print(lemmatizer.lemmatize("best", pos="a"))
print(lemmatizer.lemmatize("run"))
print(lemmatizer.lemmatize("run",'v'))

cat
cactus
goose
rock
python
good
best
run
run


In [24]:

example_sent = "This is a sample sentence, showing off the stop words filtration."
def stopWord_Lemmatize(sent, stop_words, lemmatizer ):
  word_tokens = word_tokenize(sent) # tokenize
  return_sent = "";
  
  for w in word_tokens:
      if w not in stop_words:
          return_sent +=  " " + lemmatizer.lemmatize(w) # lemmatize w before adding it to the return_sent
  return return_sent

stop_words = set(stopwords.words('english')) # NLTK 
lemmatizer = WordNetLemmatizer()

print(raw_data[0])
print(stopWord_Lemmatize(raw_data[0], stop_words, lemmatizer) ) # call the function


This movie is awesome for three main reasons. It is esthetically beautiful. I absolutely loved that. There is a bold color theme throughout the movie with extraordinary costumes and picturesque sets. A photography which looks very costly (and probably was not) completes the look . I always enjoy those stories about groups of misfits/loners coming together and becoming a family . Sometimes they fall into clichés but this one does not. This group of actors really portrays well flawed, yet extremely likable characters. Alan Larkin is the best (between him , the van and the road movie theme, I could not help but remember my favorite movie of last year Little Miss Sunshine) . I discovered Fabrizio Bentivoglio , very interesting actor, and just got annoyed a tiny little bit by Til Schweiger performance at times . The opening scene, all the scenes where they mess up their tricks are very funny. There is a mix of humor and emotion throughout the film. I like the end a lot. And of course it is

In [25]:

# pre-process the k sentences and store the result
processed_data = []

for sent in raw_data:
  processed_data.append(stopWord_Lemmatize(sent, stop_words, lemmatizer) )

#<font color = 'green'><b>Bag-of-words (BoW)

In [26]:
# Google "sklearn bag of words" ---> https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() #https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
X_BoW = count_vect.fit_transform(processed_data)

#print(count_vect.get_feature_names()) 

In [28]:
print(type(X_BoW))

<class 'scipy.sparse.csr.csr_matrix'>


In [27]:
# Sparse representations vs dense Matrices

#Refer: https://docs.scipy.org/doc/scipy/reference/sparse.html


print(X_BoW.data.nbytes) # Refer: https://stackoverflow.com/questions/43681279/why-is-scipy-sparse-matrix-memory-usage-indifferent-of-the-number-of-elements-in

# Convert X_BoW to dense
# Ref: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.todense.html#scipy.sparse.csr_matrix.todense
X_BoW_Dense = X_BoW.todense();
print(X_BoW_Dense.data.nbytes)

print(X_BoW.shape)
print(X_BoW_Dense.shape)
# Always use sparse-matrix in these situations

82968
3626400
(100, 4533)
(100, 4533)


In [29]:
print(X_BoW[0,:])

  (0, 4072)	2
  (0, 2713)	6
  (0, 361)	1
  (0, 4082)	1
  (0, 2510)	1
  (0, 3303)	1
  (0, 2197)	1
  (0, 1445)	1
  (0, 419)	1
  (0, 80)	1
  (0, 2461)	1
  (0, 4064)	2
  (0, 510)	1
  (0, 819)	1
  (0, 4061)	3
  (0, 4086)	2
  (0, 1512)	1
  (0, 944)	1
  (0, 3035)	1
  (0, 3616)	1
  (0, 3028)	1
  (0, 2442)	3
  (0, 943)	1
  (0, 3170)	1
  (0, 868)	1
  :	:
  (0, 2619)	1
  (0, 4167)	1
  (0, 1714)	1
  (0, 2677)	1
  (0, 2028)	1
  (0, 1361)	1
  (0, 1592)	1
  (0, 2400)	1
  (0, 1375)	1
  (0, 2455)	1
  (0, 226)	1
  (0, 954)	1
  (0, 2505)	2
  (0, 1797)	1
  (0, 2519)	1
  (0, 341)	1
  (0, 4381)	1
  (0, 971)	1
  (0, 2057)	1
  (0, 4436)	1
  (0, 1892)	1
  (0, 1472)	1
  (0, 1184)	1
  (0, 629)	1
  (0, 2503)	1


In [30]:
print(X_BoW[0,818])
print(X_BoW[0,817])

0
0


In [31]:
print(X_BoW_Dense[0,:])
print(X_BoW_Dense[0,818])
print(X_BoW_Dense[0,817])

[[0 0 0 ... 0 0 0]]
0
0


#<font color = 'green'><b>TD-IDF

In [32]:
# Google: "TF-IDF SkLearn" ---> https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
X_tfidf = vectorizer.fit_transform(raw_data)

#print(vectorizer.get_feature_names())

In [33]:
print(type(X_tfidf))
print(X_tfidf.shape)
print(X_tfidf.data.nbytes) # Refer: https://stackoverflow.com/questions/43681279/why-is-scipy-sparse-matrix-memory-usage-indifferent-of-the-number-of-elements-in

X_tfidf_dense = X_tfidf.todense()
print(type(X_tfidf_dense))
print(X_tfidf_dense.shape)
print(X_tfidf_dense.data.nbytes)

<class 'scipy.sparse.csr.csr_matrix'>
(100, 14902)
156768
<class 'numpy.matrix'>
(100, 14902)
11921600


#<font color = 'green'><b>Word2Vec

In [ ]:
# Gensim
# Refer: https://machinelearningmastery.com/develop-word-embeddings-python-gensim/

# Download pretrained  vectors
# https://www.quora.com/How-can-I-download-the-Google-news-word2vec-pretrained-model-from-a-Ubuntu-terminal
! wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
! gunzip GoogleNews-vectors-negative300.bin.gz


--2022-05-03 12:59:19--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.196.128
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.196.128|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-05-03 12:59:19 ERROR 404: Not Found.



In [ ]:
! ls

aclImdb  aclImdb_v1.tar.gz  sample_data


In [ ]:
# takes time as the model is large to laod into RAM
# RAM consumption also shoots up. Kernel could restart and give you a larger RAM isnatnce like 25GB RAM

from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [ ]:
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

In [ ]:
v = model['queen'];
print(type(v))
print(v.shape)

#<font color = 'green'><b>BERT


- BERT vs Word2Vec
- BERT: Contextual encodings

In [1]:
# There are many ways to obatin BERT encodings. We are using one of the 
# most simple+popular approaches
# use GPU based instance: Runtime---> Change Runtime --> GPU

# Change to TensorFlow Version 1.x 
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
#https://github.com/hanxiao/bert-as-service

# https://github.com/hanxiao/bert-as-service/issues/380
# Install BERT-SERVING client and Server
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 61 kB 242 kB/s 
     |████████████████████████████████| 357 kB 11.0 MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=9e38d59d974289fbd25ce81322f5a854d00e47f8b08b13e677479ebca94878a9
  Stored in directory: /root/.cache/pip/wheels/6e/f8/83/534c52482d6da64622ddbf72cd93c35d2ef2881b78fd08ff0c
Successfully built GPUtil


In [3]:
# dowload pretrained models
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2022-05-03 17:53:35--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.129.128, 108.177.112.128, 172.253.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.129.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   167MB/s    in 2.3s    

2022-05-03 17:53:38 (167 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [4]:
# Start BERT_SERVER on the current computer
!nohup bert-serving-start -model_dir=./uncased_L-12_H-768_A-12 > out.file 2>&1 &

In [5]:
# Use bert-client from python
# Takes time to execute as it uses GPU
from bert_serving.client import BertClient
bc = BertClient()
print (bc.encode(['First do it', 'then do it right', 'then do it better'])) # list of setences

KeyboardInterrupt: ignored

In [ ]:
v = bc.encode([raw_data[0]])

In [ ]:
print(type(v))
print(v.shape)